# 06 — Full Stack Test
End-to-end smoke test: backtest → sweep (w/ vol-carry params) → report with Pareto.
Assumes your data is in `data/prices_1d/*.parquet` and `config/baseline.yaml` exists.


In [ ]:
import os, sys, json, textwrap, pandas as pd
from pathlib import Path
root = Path.cwd()
print('CWD:', root)
assert (root/'data'/'prices_1d').exists(), 'Expect data/prices_1d with parquet files'
assert (root/'config'/'baseline.yaml').exists(), 'Expect config/baseline.yaml'


## 1) Backtest with guards + vol-carry

In [ ]:
import subprocess, sys, pandas as pd
cmd = [sys.executable, '-m', 'src.exec.backtest_pnl_demo',
       '--cfg','config/baseline.yaml', '--folder','data/prices_1d', '--costs_csv','data/costs_per_symbol.csv',
       '--target_ann_vol','0.12','--vol_lookback','20','--max_leverage','3.0',
       '--mtd_soft','-0.06','--mtd_hard','-0.10',
       '--w_tsmom','1.0','--w_xsec','0.8','--w_mr','0.6','--w_volcarry','0.4',
       '--volcarry_top_q','0.35','--volcarry_bot_q','0.35','--volcarry_lookback','63',
       '--gap_atr_k','3.0','--atr_lookback','14','--vol_spike_mult','3.0','--vol_spike_window','60']
print('RUN:', ' '.join(cmd))
import subprocess as sp
sp.check_call(cmd)
pd.read_csv('data/pnl_demo_equity.csv').head()


## 2) Small robustness sweep (fast)

In [ ]:
import subprocess as sp, sys
out_root = 'runs/robustness_quickdemo'
cmd = [sys.executable, '-m', 'src.exec.sweep_robustness',
       '--cfg','config/baseline.yaml','--folder','data/prices_1d','--costs_csv','data/costs_per_symbol.csv',
       '--w_volcarry_grid','0.0,0.3', '--volcarry_topq_grid','0.25,0.45', '--volcarry_botq_grid','0.25,0.45', '--volcarry_lookback_grid','42,84',
       '--target_vol_grid','0.10,0.15','--vol_lookback_grid','15,30','--max_lev_grid','2.0,3.0', '--limit','16',
       '--out_root', out_root]
print('RUN:', ' '.join(cmd))
sp.check_call(cmd)
from pathlib import Path
assert Path(out_root,'summary.csv').exists(), 'summary.csv not created'
import pandas as pd
pd.read_csv(Path(out_root,'summary.csv')).head()


## 3) Build report with Pareto table

In [ ]:
import subprocess as sp, sys
out_html = 'reports/report_quickdemo.html'
cmd = [sys.executable, '-m', 'src.exec.make_report_html',
       '--equity_csv','data/pnl_demo_equity.csv','--attrib_csv','data/pnl_demo_attrib_sleeve.csv',
       '--monthly_csv','reports/monthly_summary.csv','--robustness_summary_csv','runs/robustness_quickdemo/summary.csv',
       '--pareto_top_n','8','--out_html', out_html]
print('RUN:', ' '.join(cmd))
sp.check_call(cmd)
print('Report at', out_html)
